In [6]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import os
import datetime
from pathlib import Path

In [7]:
#Selecionando as Variáveis
ano_dt_base = 2020
tx_juros = 0.06
cres_sal = 0.01
cres_benef = 0.01

Colunas que vai precisar pro cálculo:

LOTAÇÃO	MATRICULA	GRUPO	CRITERIO DE APOSENTADORIA	SEXO	DTNASC	DTADM	DATACARGO	SALPART	CARREIRA	TSANT	CONJUGE	DTNASCONJ	DTNASDEPINV	DTNASDEPVAL

In [8]:
#Importando as bases
bd_ativos = pd.read_csv(str(Path(os.getcwd())) + '\\1.SERVIDORES.csv', encoding='UTF-8', sep=';')#, thousands=',')

#Convertendo os dados para float
bd_ativos['DT_NASC_SERVIDOR'] = pd.to_datetime(bd_ativos['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_ENTE'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_CARGO'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
bd_ativos['DT_NASC_CONJUGE'] = pd.to_datetime(bd_ativos['DT_NASC_CONJUGE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_NOVO'] = pd.to_datetime(bd_ativos['DT_NASC_NOVO'], yearfirst=True, format='%d/%m/%Y')

In [9]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = ano_dt_base - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Ano da Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_ADM'] =  bd_ativos['DT_ING_ENTE'].dt.strftime('%Y').astype(int) - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos['DT_NASC_CONJUGE'].mask(bd_ativos['DT_NASC_CONJUGE'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_CONJUGE'] = ano_dt_base - statis_ativos['IDADE_CONJUGE'].dt.strftime('%Y').astype(int) #Calculando as Idades dos Cônjuges | Ano da Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 1) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 3) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 4) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 5), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos['DT_NASC_NOVO'].mask(bd_ativos['DT_NASC_NOVO'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_NOVO'] = ano_dt_base - statis_ativos['IDADE_NOVO'].dt.strftime('%Y').astype(int) #Calculando as Idades do Filho Mais Novo | Ano da Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos['IDADE_NOVO'] > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos['IDADE_CONJUGE'] - statis_ativos['IDADE_SERVIDOR'] #Calculando a Diferença Etária do Cônjuge

#statistics_ativos['TSANT'] = bd_ativos['DT_ING_ENTE'] - pd.to_timedelta(bd_ativos['NU_TEMPO_RGPS'], unit='d')
statis_ativos.head(15)

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE,IDADE_NOVO,DIF_ETARIA
0,40,20,NaN,16.0,NaN
1,45,42,48.0,NaN,3.0
2,48,27,NaN,18.0,NaN
3,31,24,NaN,NaN,NaN
4,42,35,NaN,NaN,NaN
5,51,29,NaN,NaN,NaN
6,40,22,53.0,NaN,13.0
7,40,34,NaN,11.0,NaN
8,45,42,NaN,NaN,NaN
9,41,31,NaN,NaN,NaN


In [10]:
#bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
#bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].astype(float)
#bd_ativos['TESTE'] = bd_ativos['VL_BASE_CALCULO'] + 1000
bd_ativos['VL_BASE_CALCULO']

0      1100.00
1      1100.00
2      1708.84
3      3378.49
4      1100.00
        ...   
357    2410.58
358    2972.73
359    3326.21
360    2141.74
361    3426.86
Name: VL_BASE_CALCULO, Length: 362, dtype: float64